# Trabajo Práctico 1: Análisis Exploratorio de Datos #
## Cientes potenciales a tarjetas de crédito. ##
El [DATASET](https://www.kaggle.com/datasets/sajidhussain3/jobathon-may-2021-credit-card-lead-prediction?select=train.csv) contiene información de clientes de un banco denominado Happy Customer Bank. Es un banco privado de tamaño medio que se ocupa de todo tipo de productos bancarios, como cuentas de ahorro, cuentas corrientes, productos de inversión, productos de crédito, entre otras ofertas.  

En este caso, el Happy Customer Bank desea realizar ventas cruzadas de sus tarjetas de crédito a sus clientes existentes. El banco ha identificado un conjunto de clientes que son elegibles para aceptar estas tarjetas de crédito.

El banco tiene la necesidad, de identificar a los clientes que podrían mostrar intención hacia una tarjeta de crédito recomendada.

<img src = "https://storage.googleapis.com/kaggle-datasets-images/1371711/2277422/2c35585d42b3747882e83e926d0f643e/dataset-cover.png?t=2021-05-27-18-40-42">


 --- 

## 1. Listado de variables y selección ###
### a) Variables de Entrada ##

- **ID:** Identificador único para cada fila. (no lo vamos a usar)



- **Gender:** Género del cliente.



- **Age:** Edad del cliente (en años).


        
- **Region_Code:** Código de la Región para los clientes


        
- **Occupation:** Ocupación del cliente.

        
        
- **Channel_Code:** Código de Canal de Adquisición para el Cliente (Codificado)


        
- **Vintage:** Antiguedad del Cliente (En Meses).

        
        
- **Credit_Product:** Si el Cliente tiene algún producto de crédito activo (Préstamo de vivienda, Préstamo personal, Tarjeta de crédito, etc.)



- **Avg_Account_Balance:** Saldo de cuenta promedio. Saldo promedio de la cuenta del cliente en los últimos 12 meses

      
      
- **Is_Active:** Si el Cliente tiene activo algún producto de crédito. Si el cliente está activo en los últimos 3 meses. 
            0-> El cliente no esta activo | 1-> el cliente esta activo
            
            
--------

### b) Variable de Salida ###

- **Is_Lead:**  Si el Cliente está interesado en la Tarjeta de Crédito.
            0-> El cliente no esta interesado | 1-> el cliente esta interesado



In [2]:
import numpy as np
import pandas as pd 
import matplotlib
import keras
import h5py
import PIL
import seaborn
import sklearn
import pytz

from matplotlib import pyplot as plt

from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

pd.options.display.max_columns = 0

plt.rcParams.update({
    "font.family": ["serif"],
    "font.sans-serif": ["Roboto"],
    "font.size": 9,
    "axes.labelsize": 11,
    "axes.titlesize": 13,
    "xtick.labelsize": 11,
    "ytick.labelsize": 11,
    "legend.fontsize": 11,
    'figure.figsize': (15.0, 4.0),
    'axes.grid': False,
    'axes.spines.left': True,
    'axes.spines.right': True,
    'axes.spines.top': True,
    'axes.spines.bottom': True,
})

np.set_printoptions(suppress=True)

#warnings.filterwarnings('ignore')

In [3]:
passes = pd.read_csv('train.csv')

In [4]:
passes.sample(10)

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
61934,QSM2MB2W,Female,29,RG257,Self_Employed,X1,15,No,438262,No,0
5589,KJOZANYF,Male,29,RG263,Other,X1,14,Yes,809481,No,0
13748,GCDECXHZ,Male,30,RG274,Other,X1,33,No,3729464,No,0
29992,KYESAOVG,Male,48,RG254,Self_Employed,X2,50,No,1745263,Yes,0
127368,4ANZZBCU,Male,77,RG270,Other,X4,7,No,934624,Yes,0
61756,JF4HMQGJ,Female,54,RG284,Self_Employed,X3,93,NaN,2172169,Yes,1
115092,XQ5YLFAD,Male,72,RG254,Other,X3,117,No,2362887,Yes,0
159210,QJBQKEXG,Female,71,RG284,Other,X3,56,No,1279375,No,1
120218,5V95GAHK,Female,60,RG283,Self_Employed,X2,68,NaN,1188541,Yes,1
67398,RBGHNW8M,Female,29,RG254,Salaried,X1,27,No,457771,No,0


### c) Transformaciones ###

In [ ]:
# 1. Obtener los nombres de las columas como una lista
col_name = passes.columns.tolist()

# 2. Itera sobre la lista de nombres
for column in col_name:
    print ("Valores nulos en <{0}>: {1}".format(column, passes[column].isnull().sum()))

In [ ]:
Nulos = passes.isnull().sum()
print("Columnas         Cantidad de nulos")
print(Nulos)

In [ ]:
passes.count()

- **Gender:** one-hot encoder



- **Age:** rango


        
- **Region_Code:** one-hot encoder


        
- **Occupation:** one-hot encoder

        
        
- **Channel_Code:** one-hot encoder


        
- **Vintage:** rango

        
        
- **Credit_Product:** one-hot encoder y eliminamos los null. (esta desbalanceado)



- **Avg_Account_Balance:** escalar y rangos (y ver outliers)

      
      
- **Is_Active:** one-hot encoder

In [ ]:
# Se dividen los datos en train y test
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(passes, test_size=0.4)
validation, test = train_test_split(not_train, test_size=0.5)

#Dimención (filas, columnas) de los distintos dataset
print (" Train:", train.shape, '\n', 'Validation:', validation.shape, '\n', 'Test:', test.shape)
#print("first line", "second line", sep="\n")`